# Introduction to Data Science – Homework 6
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, February 23, 11:59pm.

In Part 1 of this homework you will scrape github repositories and organize the information in a Pandas dataframe. In Part 2, you will use linear regression to gain meaningful insights. 

## Your Data
First Name:
<br>
Last Name:
<br>
E-mail:
<br>
UID:
<br>

In [1]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request


import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

### 1. Scrape Github Repository List and Repository Information using BeautifulSoup
In this part you will explore over 2 million Github repositories. You are going to scrape [this repository list](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories).

### 1.1. Check whether you are permitted to scrape the data
Before you start to scrape any website you should go through the terms and services or similar pages of the website. Almost all websites post conditions to use their data. Check the terms of [https://github.com/](https://github.com/) to see whether the site permits you to scrape their data or not. Are you sure you are allowed to scrape?

**Your Interpretation:**
TODO

### Task 1.2 Download the website

Download the **first ten** pages of the list of highly starred repositories here: 

https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories

To avoid sending too many requests to the server, make sure that your querying code is in a sperate cell from anything else, so that you don't have to re-run it needlessly. 

**Warning:** while we haven't experienced any problems, exepct that github will temporarily block access if you scrape their website relentlessly. Make sure to be responsible. Also, don't do this last minute – if you're blocked for e.g., 24 hours that could impact your timeline.

Now read the html text in python and create a soup object.

In [14]:
## Your code goes here

# download the fist ten pages
url = "https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=stars&type=Repositories"

# with urllib.request.urlopen(url) as response:
#     html = response.read()
#     html = html.decode('utf-8')

#A simple solution is to just use ‘response.text’ to get html returned by requests.get(url) directly.
    
html = requests.get(url)
html = html.text

# save the file
with open('git_repositories.html', 'w') as new_file:
    new_file.write(html)

# here it's already a local operation
git_soup = BeautifulSoup(html, 'html.parser')

UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4da' in position 32923: character maps to <undefined>

In [2]:
# Creating the soup

### 1.3. Extract Data

Extract the foolowing data for each repository, and create a Pandas Dataframe with a row for each repository and a column for each of these datums. 

1. Name of the repository
+ URL of the repository 
+ Number of Contributors 
+ Programming Language Used 
+ Number of stars 
+ Number of issues
+ Number of files 
+ Length of readme file.

Note that you will frist have to extract links from the soup that you just scraped earlier, and then download the repository pages to retreive most of the data. 

Refer to lecture 12 for details on how to do this. Make sure to use the web inspector to identify the relevant structures. 

Save the dataframe you created to a new file `project_info.csv` and include this in your submission.

In [3]:
## Your code goes here

In [4]:
# Convert list into DataFrame

In [6]:
# Save dataframe to file project_info.csv

### 2. Use linear regression to analyze the Github repository data

In this part, you will analyze the data collectd in Part 1 using regression tools. The goal is to identify properties that make a repository popular. 

First, load the project_info.csv file in again. **We need you to do this so that we can run your code below withouth having to run your scraping code.**

In [7]:
# this loads the data from the project_info.csv file 
project_info = pd.read_csv('project_info.csv')
project_info.head()

,Repository Name,Language,Stars,URL,Forks,Contributors,Issues,ReadmeLength
0,freeCodeCamp/freeCodeCamp,JavaScript,292k,https://github.com/freeCodeCamp/freeCodeCamp,"8,409",731,165,2798
1,twbs/bootstrap,CSS,122k,https://github.com/twbs/bootstrap,"7,279",982,312,5032
2,EbookFoundation/free-programming-books,NaN,101k,https://github.com/EbookFoundation/free-progra...,"7,387",978,16,1586
3,tensorflow/tensorflow,C++,89.7k,https://github.com/tensorflow/tensorflow,"7,449","1,337","1,194",3028
4,facebook/react,JavaScript,88.5k,https://github.com/facebook/react,"5,521","1,167",320,3311


### 2.1. Reformat the data into useful datatypes

1. Print the data types for the columns of your dataframe
+ Reformat the colums Issues, Forks, Stars, and Contributers to be integer data
+ There is one repository flagged as having infinite contributers (the Linux kernel). We'll assume that it in fact has 15000 contributors (about twice as much as the next project). 

In [5]:
# your code goes here


### Task 2.2 Describe the data

+ Get an overview of the data using the describe function.
+ Compute the correlation matrix, visualize it with a heat map.
+ Visualize the correlations by making a scatterplot matrix.
+ Interprete what you see.

You can re-use code from your previous homework here.

In [6]:
# your code goes here

**Your Interpretation:** TODO

### 2.3. Linear regression

1. First use linear regression to try to predict the number of Stars based on Forks, Contributors, Issues, and Readme Length. Explain why this is not a very good model by discussing the R-squared , F-statistic p-value, and coefficient  p-values. 
+ Develop another model which is better. Explain why it is better and interpret your results. 

In [7]:
# your code goes here

** Your interpretation:** TODO